<a href="https://colab.research.google.com/github/oreomcflurryyy/statistical-learning-deeplearning/blob/main/deeplearning_9-10-11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ISLP
!pip install pytorch-lightning
!pip install torchmetrics
!pip install torchinfo

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 501.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.0/522.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 2.7 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=8f89baab45e4f2d90ed56706e794b8f1290c7e9b065248e791b29f00eb5b11e0
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma
  Attempting u

In [38]:
import numpy as np, pandas as pd
from matplotlib.pyplot import subplots
from sklearn.linear_model import \
     (LinearRegression,
      LogisticRegression,
      Lasso)
from sklearn.metrics import (r2_score, mean_squared_error)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from sklearn.model_selection import \
     (train_test_split,
      GridSearchCV)

In [3]:
import torch
from torch import nn
from torch.optim import RMSprop
from torch.utils.data import TensorDataset

In [4]:
from torchmetrics import (MeanAbsoluteError,
                          R2Score)
from torchinfo import summary

In [5]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger

In [6]:
from pytorch_lightning import seed_everything
seed_everything(0, workers=True)
torch.use_deterministic_algorithms(True, warn_only=True)

INFO:lightning_fabric.utilities.seed:Seed set to 0


In [7]:
from torchvision.io import read_image
from torchvision.datasets import MNIST, CIFAR100
from torchvision.models import (resnet50,
                                ResNet50_Weights)
from torchvision.transforms import (Resize,
                                    Normalize,
                                    CenterCrop,
                                    ToTensor)

In [8]:
from ISLP.torch import (SimpleDataModule,
                        SimpleModule,
                        ErrorTracker,
                        rec_num_workers)

In [9]:
from ISLP.torch.imdb import (load_lookup,
                             load_tensor,
                             load_sparse,
                             load_sequential)

In [10]:
from glob import glob
import json

Fit a lag-5 autoregressive model to the NYSE data, as described in
the text and Lab 10.9.6. Refit the model with a 12-level factor repre-
senting the month.

In [11]:
NYSE = load_data('NYSE')
NYSE = NYSE.reset_index()
NYSE['month'] = NYSE['date'].str[5:7]

cols = ['month','DJ_return', 'log_volume', 'log_volatility']
X = pd.DataFrame(StandardScaler(
                     with_mean=True,
                     with_std=True).fit_transform(NYSE[cols]),
                 columns=NYSE[cols].columns,
                 index=NYSE.index)
NYSE

,date,day_of_week,DJ_return,log_volume,log_volatility,train,month
0,1962-12-03,mon,-0.004461,0.032573,-13.127403,True,12
1,1962-12-04,tues,0.007813,0.346202,-11.749305,True,12
2,1962-12-05,wed,0.003845,0.525306,-11.665609,True,12
3,1962-12-06,thur,-0.003462,0.210182,-11.626772,True,12
4,1962-12-07,fri,0.000568,0.044187,-11.728130,True,12
...,...,...,...,...,...,...,...
6046,1986-12-24,wed,0.006514,-0.236104,-9.807366,False,12
6047,1986-12-26,fri,0.001825,-1.322425,-9.906025,False,12
6048,1986-12-29,mon,-0.009515,-0.371237,-9.827660,False,12
6049,1986-12-30,tues,-0.001837,-0.385638,-9.926091,False,12


In [12]:
for lag in range(1, 6):
    for col in cols:
        newcol = np.zeros(X.shape[0]) * np.nan
        newcol[lag:] = X[col].values[:-lag]
        X.insert(len(X.columns), "{0}_{1}".format(col, lag), newcol)
X.insert(len(X.columns), 'train', NYSE['train'])
X = X.dropna()

In [13]:
Y, train = X['log_volume'], X['train']
X = X.drop(columns=['train'] + cols)
X.columns

Index(['month_1', 'DJ_return_1', 'log_volume_1', 'log_volatility_1', 'month_2',
       'DJ_return_2', 'log_volume_2', 'log_volatility_2', 'month_3',
       'DJ_return_3', 'log_volume_3', 'log_volatility_3', 'month_4',
       'DJ_return_4', 'log_volume_4', 'log_volatility_4', 'month_5',
       'DJ_return_5', 'log_volume_5', 'log_volatility_5'],
      dtype='object')

In [14]:
M = LinearRegression()
M.fit(X[train], Y[train])
M.score(X[~train], Y[~train])

0.41710083803789133

In [15]:
X_day = pd.concat([X,
                  pd.get_dummies(NYSE['day_of_week'])],
                  axis=1).dropna()
X_day.columns

Index(['month_1', 'DJ_return_1', 'log_volume_1', 'log_volatility_1', 'month_2',
       'DJ_return_2', 'log_volume_2', 'log_volatility_2', 'month_3',
       'DJ_return_3', 'log_volume_3', 'log_volatility_3', 'month_4',
       'DJ_return_4', 'log_volume_4', 'log_volatility_4', 'month_5',
       'DJ_return_5', 'log_volume_5', 'log_volatility_5', 'mon', 'tues', 'wed',
       'thur', 'fri'],
      dtype='object')

In [16]:
M.fit(X_day[train], Y[train])
M.score(X_day[~train], Y[~train])

0.4640149323946233

In [40]:
y_pred = M.predict(X_day[~train])
test_r2 = r2_score(Y[~train], y_pred)
print(f"Test R^2: {test_r2}")

test_loss = mean_squared_error(Y[~train], y_pred)
print(f"Test Loss (MSE): {test_loss}")

Test R^2: 0.4640149323946233
Test Loss (MSE): 0.5647602743945903


In [17]:
ordered_cols = []
for lag in range(5,0,-1):
    for col in cols:
        ordered_cols.append('{0}_{1}'.format(col, lag))
X = X.reindex(columns=ordered_cols)
X.columns

Index(['month_5', 'DJ_return_5', 'log_volume_5', 'log_volatility_5', 'month_4',
       'DJ_return_4', 'log_volume_4', 'log_volatility_4', 'month_3',
       'DJ_return_3', 'log_volume_3', 'log_volatility_3', 'month_2',
       'DJ_return_2', 'log_volume_2', 'log_volatility_2', 'month_1',
       'DJ_return_1', 'log_volume_1', 'log_volatility_1'],
      dtype='object')

In [18]:
X_rnn = X.to_numpy().reshape((-1,5,4))
X_rnn.shape

(6046, 5, 4)

In [19]:
class NYSEModel(nn.Module):
    def __init__(self):
        super(NYSEModel, self).__init__()
        self.rnn = nn.RNN(4,
                          12,
                          batch_first=True)
        self.dense = nn.Linear(12, 1)
        self.dropout = nn.Dropout(0.1)
    def forward(self, x):
        val, h_n = self.rnn(x)
        val = self.dense(self.dropout(val[:,-1]))
        return torch.flatten(val)
nyse_model = NYSEModel()

In [20]:
datasets = []
for mask in [train, ~train]:
    X_rnn_t = torch.tensor(X_rnn[mask].astype(np.float32))
    Y_t = torch.tensor(Y[mask].to_numpy().astype(np.float32))
    datasets.append(TensorDataset(X_rnn_t, Y_t))
nyse_train, nyse_test = datasets

In [21]:
summary(nyse_model,
        input_data=X_rnn_t,
        col_names=['input_size',
                   'output_size',
                   'num_params'])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
NYSEModel                                [1770, 5, 4]              [1770]                    --
├─RNN: 1-1                               [1770, 5, 4]              [1770, 5, 12]             216
├─Dropout: 1-2                           [1770, 12]                [1770, 12]                --
├─Linear: 1-3                            [1770, 12]                [1770, 1]                 13
Total params: 229
Trainable params: 229
Non-trainable params: 0
Total mult-adds (M): 1.93
Input size (MB): 0.14
Forward/backward pass size (MB): 0.86
Params size (MB): 0.00
Estimated Total Size (MB): 1.01

In [22]:
max_num_workers = rec_num_workers()
nyse_dm = SimpleDataModule(nyse_train,
                           nyse_test,
                           num_workers=min(4, max_num_workers),
                           validation=nyse_test,
                           batch_size=64)

In [23]:
for idx, (x, y) in enumerate(nyse_dm.train_dataloader()):
    out = nyse_model(x)
    print(y.size(), out.size())
    if idx >= 2:
        break

torch.Size([64]) torch.Size([64])
torch.Size([64]) torch.Size([64])
torch.Size([64]) torch.Size([64])


In [24]:
nyse_optimizer = RMSprop(nyse_model.parameters(),
                         lr=0.001)
nyse_module = SimpleModule.regression(nyse_model,
                                      optimizer=nyse_optimizer,
                                      metrics={'r2':R2Score()})

In [25]:
nyse_trainer = Trainer(deterministic=True,
                       max_epochs=200,
                       enable_progress_bar=False,
                       callbacks=[ErrorTracker()])
nyse_trainer.fit(nyse_module,
                 datamodule=nyse_dm)
nyse_trainer.test(nyse_module,
                  datamodule=nyse_dm)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type      | Params | Mode 
--------------------------------------------
0 | model | NYSEModel | 229    | train
1 | loss  | MSELoss   | 0      | train
--------------------------------------------
229       Trainable params
0         Non-trainable params
229       Total params
0.001     Total estimated model params size (MB)
5         Modules in train mode
0         Modules in eval mode
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=200` reached.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.6212008595466614     │
│          test_r2          │    0.4104501008987427     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.6212008595466614, 'test_r2': 0.4104501008987427}]

Does this factor improve the performance of the
model?

It does not improve the performance, and there is marginal poorer performance instead. This can indicate that `month` has little to no significance in getting the target variable. We find in both Linear Regression and in lag-5 autoregressive neural network, incorporating the `day_of_week` has improved the model.

---

In Section 10.9.6, we showed how to fit a linear AR model to the
NYSE data using the LinearRegression( ) function. However, we also
mentioned that we can “flatten” the short sequences produced for
the RNN model in order to fit a linear AR model. Use this latter
approach to fit a linear AR model to the NYSE data.

In [26]:
datasets = []
for mask in [train, ~train]:
    X_day_t = torch.tensor(
                   np.asarray(X_day[mask]).astype(np.float32))
    Y_t = torch.tensor(np.asarray(Y[mask]).astype(np.float32))
    datasets.append(TensorDataset(X_day_t, Y_t))
day_train, day_test = datasets

In [27]:
day_dm = SimpleDataModule(day_train,
                          day_test,
                          num_workers=min(4, max_num_workers),
                          validation=day_test,
                          batch_size=64)

In [34]:
class LinearARModel(nn.Module):
    def __init__(self):
        super(LinearARModel, self).__init__()
        self._forward = nn.Sequential(nn.Flatten(),
                                      nn.Linear(25, 32),
                                      nn.Dropout(0.5),
                                      nn.Linear(32, 1))
    def forward(self, x):
        return torch.flatten(self._forward(x))

In [35]:
l_model = LinearARModel()
l_optimizer = RMSprop(l_model.parameters(),
                           lr=0.001)
l_module = SimpleModule.regression(l_model,
                                        optimizer=l_optimizer,
                                        metrics={'r2':R2Score()})

In [36]:
l_trainer = Trainer(deterministic=True,
                     max_epochs=20,
                     enable_progress_bar=False,
                     callbacks=[ErrorTracker()])
l_trainer.fit(l_module, datamodule=day_dm)
l_trainer.test(l_module, datamodule=day_dm)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type          | Params | Mode 
------------------------------------------------
0 | model | LinearARModel | 865    | train
1 | loss  | MSELoss       | 0      | train
------------------------------------------------
865       Trainable params
0         Non-trainable params
865       Total params
0.003     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.5764352679252625     │
│          test_r2          │    0.45293480157852173    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.5764352679252625, 'test_r2': 0.45293480157852173}]

Compare the test R2 of this linear AR model to that of the linear AR model that we fit in the lab.

### Comparison and Analysis

#### Test \( R^2 \) and Test Loss (MSE)

- **Traditional Linear AR Model**:
  - Test \( R^2 \): 0.4640
  - Test Loss (MSE): 0.5648

- **Deep Learning Linear AR Model**:
  - Test \( R^2 \): 0.4529
  - Test Loss (MSE): 0.5764

The traditional linear AR model slightly outperforms the deep learning linear AR model in terms of both \( R^2 \) and MSE. This suggests that for this particular dataset and task, the simpler traditional model is more effective.


What are the advantages/disadvantages of each approach?

| **Aspect**            | **Traditional Linear AR Model**                                      | **Deep Learning Linear AR Model**                                      |
|-----------------------|----------------------------------------------------------------------|------------------------------------------------------------------------|
| **Simplicity**        | Easier to implement and interpret.                                   | More complex to implement and tune.                                    |
| **Efficiency**        | Requires less computational power and memory.                        | Generally requires more computational resources.                       |
| **Interpretability**  | Parameters are straightforward to understand.                        | Less interpretable due to complex architectures.                       |
| **Flexibility**       | Limited to capturing linear relationships.                           | Can be extended to include non-linear components and complex patterns. |
| **Feature Engineering** | Requires manual feature engineering.                               | Can automatically learn features through layers.                       |
| **Integration**       | Standalone, simpler integration.                                     | Fits well into modern ML pipelines and can be combined with other models. |
| **Scalability**       | Suitable for smaller datasets.                                       | Can handle large datasets if designed appropriately.                   |
| **Overfitting Risk**  | Lower risk of overfitting with simpler models.                       | Higher risk of overfitting without proper regularization.              |

---

 Repeat the previous exercise, but now fit a nonlinear AR model by
“flattening” the short sequences produced for the RNN model.

In [28]:
class NonLinearARModel(nn.Module):
    def __init__(self):
        super(NonLinearARModel, self).__init__()
        self._forward = nn.Sequential(nn.Flatten(),
                                      nn.Linear(25, 32),
                                      nn.ReLU(),
                                      nn.Dropout(0.5),
                                      nn.Linear(32, 1))
    def forward(self, x):
        return torch.flatten(self._forward(x))

In [29]:
nl_model = NonLinearARModel()
nl_optimizer = RMSprop(nl_model.parameters(),
                           lr=0.001)
nl_module = SimpleModule.regression(nl_model,
                                        optimizer=nl_optimizer,
                                        metrics={'r2':R2Score()})

In [30]:
nl_trainer = Trainer(deterministic=True,
                     max_epochs=20,
                     enable_progress_bar=False,
                     callbacks=[ErrorTracker()])
nl_trainer.fit(nl_module, datamodule=day_dm)
nl_trainer.test(nl_module, datamodule=day_dm)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params | Mode 
---------------------------------------------------
0 | model | NonLinearARModel | 865    | train
1 | loss  | MSELoss          | 0      | train
---------------------------------------------------
865       Trainable params
0         Non-trainable params
865       Total params
0.003     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.5621899366378784     │
│          test_r2          │    0.46645426750183105    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.5621899366378784, 'test_r2': 0.46645426750183105}]